In [2]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from sklearn import preprocessing

In [3]:
ynames = [
	"time_steps",
	"state_depth",
	"filters_downsampling",
	"downsampling_layers",
	"conv_layers",
	"hidden_layers",
	"filters_upsampling",
	"kernel_regularizer_amp",
	"bias_regularizer_amp",
	"batch_norm",
	"activation",
	"initial_learning_rate",
	"beta_1",  # Adam update in RIM
	"beta_2",
    "train_loss",
    "chi_squared_train"
    ]

df = pd.read_csv("../results/scores_1.csv", names=ynames)
df2 = pd.read_csv("../results/scores_2.csv", names=ynames)
df3 = pd.read_csv("../results/scores_3.csv", names=ynames)
df = pd.concat([df, df2, df3])
le = preprocessing.LabelEncoder()
le.fit(df["activation"])
df["activation"] = le.transform(df["activation"])
le.classes_

array(['elu', 'gelu', 'leaky_relu', 'relu'], dtype=object)

In [4]:
fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = df['train_loss'],
                   colorscale = 'Jet',
                   showscale = True,
                   cmin = 0,
                   cmax = 0.6),
        dimensions = list([
            dict(range = [6, 12],
                 # constraintrange = [100000,150000],
                 label = "Time steps", 
                 values = df['time_steps']),
            dict(label= "State Depth",
                values=df["state_depth"]),
#             dict(label="filters_downsampling", values=df["filters_downsampling"]),
#             dict(label="downsampling_layers", values=df["downsampling_layers"]),
#             dict(label="conv_layers", values=df["conv_layers"]),
#             dict(label="hidden_layers", values=df["hidden_layers"]),
#             dict(label="filters_upsampling", values=df["filters_upsampling"]),
            dict(label="batch_norm", values=df["batch_norm"].to_numpy().astype(int), 
                ticktext = ["True", "False"], tickvals=[1, 0]),
            dict(label="activation", values=df["activation"], 
                ticktext=list(le.classes_), tickvals=list(range(4))),
            dict(label="Learning rate", values=df["initial_learning_rate"]),
            dict(label="beta_1", values=df["beta_1"]),
            dict(label="beta_2", values=df["beta_2"]),
            # dict(range = [0,700000],
                 # label = 'Block Width', values = df['blockWidth']),
            # dict(tickvals = [0,0.5,1,2,3],
                 # ticktext = ['A','AB','B','Y','Z'],
                 # label = 'Cyclinder Material', values = df['cycMaterial']),
            # dict(range = [-1,4],
                 # tickvals = [0,1,2,3],
                 # label = 'Block Material', values = df['blockMaterial']),
            # dict(range = [134,3154],
                 # visible = True,
                 # label = 'Total Weight', values = df['totalWeight']),
            # dict(range = [9,19984],
                 # label = 'Assembly Penalty Wt', values = df['assemblyPW']),
            # dict(range = [49000,568000],
                 # label = 'Height st Width', values = df['HstW'])
            dict(label = "Loss", 
                constraintrange = [0, 0.5],
                values = df["train_loss"]),
            ])
    )
)
fig.write_html('first_figure.html', auto_open=True)

In [5]:
df.sort_values("train_loss")

,time_steps,state_depth,filters_downsampling,downsampling_layers,conv_layers,hidden_layers,filters_upsampling,kernel_regularizer_amp,bias_regularizer_amp,batch_norm,activation,initial_learning_rate,beta_1,beta_2,train_loss,chi_squared_train
0,10,256,64,2,2,2,32,0.001,0.100,True,0,0.0010,0.72,0.811,0.008084,2.059645e+32
0,10,128,64,2,2,2,16,0.001,0.001,True,0,0.0010,0.71,0.989,0.008427,1.232395e+32
0,10,256,32,2,1,2,64,0.001,0.001,True,0,0.0010,0.72,0.817,0.009381,2.305900e+33
0,12,256,64,2,2,2,32,0.001,0.001,False,0,0.0010,0.87,0.981,0.009390,1.060809e+32
0,8,128,16,2,2,1,64,0.001,0.010,True,0,0.0010,0.83,0.836,0.009925,1.418931e+34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,8,256,16,2,1,2,32,0.010,0.100,False,0,0.0100,0.73,0.915,1.634157,8.728843e+09
0,12,256,32,2,1,1,64,0.010,0.100,False,2,0.0100,0.79,0.952,2.689401,NaN
0,8,32,16,2,2,1,64,0.100,0.001,False,0,0.0100,0.90,1.000,NaN,NaN
0,10,256,16,2,1,2,16,0.010,0.001,True,2,0.0001,0.66,1.000,NaN,1.269293e+10


In [44]:
best = {ynames[i]: df.sort_values("train_loss").iloc[2, i] for i in range(len(ynames) - 2)}
best["activation"] = le.classes_[best["activation"]]
for key, value in best.items(): # convert types back to basic types since json is complaining
    if isinstance(value, np.int64):
        value = int(value)
        best[key] = value
    if isinstance(value, np.bool_):
        value = bool(value)
        print(value)
        best[key] = value
best

True


{'time_steps': 10,
 'state_depth': 256,
 'filters_downsampling': 32,
 'downsampling_layers': 2,
 'conv_layers': 1,
 'hidden_layers': 2,
 'filters_upsampling': 64,
 'kernel_regularizer_amp': 0.001,
 'bias_regularizer_amp': 0.001,
 'batch_norm': True,
 'activation': 'elu',
 'initial_learning_rate': 0.001,
 'beta_1': 0.72,
 'beta_2': 0.817}

In [46]:
import json
with open("../best_hparams.json", "w") as fp:
    json.dump(best, fp)

In [47]:
with open("../best_hparams.json", "r") as fp:
    h = json.load(fp)

In [48]:
h

{'time_steps': 10,
 'state_depth': 256,
 'filters_downsampling': 32,
 'downsampling_layers': 2,
 'conv_layers': 1,
 'hidden_layers': 2,
 'filters_upsampling': 64,
 'kernel_regularizer_amp': 0.001,
 'bias_regularizer_amp': 0.001,
 'batch_norm': True,
 'activation': 'elu',
 'initial_learning_rate': 0.001,
 'beta_1': 0.72,
 'beta_2': 0.817}